# Supervised Neural Nets

In [1]:
# Import modules.
import time
import pandas as pd
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.model_selection import cross_val_score
from sklearn.neural_network import MLPClassifier

In [2]:
# Load data.
raw_data = pd.read_csv('https://media.githubusercontent.com/media/MuseumofModernArt/collection/master/Artworks.csv')

In [3]:
def viewDf(dataframe):
    """Prints general dataframe info."""
    print('General info:')
    print(dataframe.info())
    print('\nNull Counts:\n', dataframe.isnull().sum())
    print('\nStatistics:\n', dataframe.describe())
    print('\nViewing...')
    return dataframe.head()

viewDf(raw_data)

General info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 136759 entries, 0 to 136758
Data columns (total 29 columns):
Title                 136720 non-null object
Artist                135301 non-null object
ConstituentID         135301 non-null object
ArtistBio             131293 non-null object
Nationality           135301 non-null object
BeginDate             135301 non-null object
EndDate               135301 non-null object
Gender                135301 non-null object
Date                  134348 non-null object
Medium                125694 non-null object
Dimensions            125749 non-null object
CreditLine            133870 non-null object
AccessionNumber       136759 non-null object
Classification        136759 non-null object
Department            136759 non-null object
DateAcquired          129922 non-null object
Cataloged             136759 non-null object
ObjectID              136759 non-null int64
URL                   79218 non-null object
ThumbnailURL          

,Title,Artist,ConstituentID,ArtistBio,Nationality,BeginDate,EndDate,Gender,Date,Medium,...,ThumbnailURL,Circumference (cm),Depth (cm),Diameter (cm),Height (cm),Length (cm),Weight (kg),Width (cm),Seat Height (cm),Duration (sec.)
0,"Ferdinandsbrücke Project, Vienna, Austria, Ele...",Otto Wagner,6210,"(Austrian, 1841–1918)",(Austrian),(1841),(1918),(Male),1896,Ink and cut-and-pasted painted pages on paper,...,http://www.moma.org/media/W1siZiIsIjU5NDA1Il0s...,NaN,NaN,NaN,48.6000,NaN,NaN,168.9000,NaN,NaN
1,"City of Music, National Superior Conservatory ...",Christian de Portzamparc,7470,"(French, born 1944)",(French),(1944),(0),(Male),1987,Paint and colored pencil on print,...,http://www.moma.org/media/W1siZiIsIjk3Il0sWyJw...,NaN,NaN,NaN,40.6401,NaN,NaN,29.8451,NaN,NaN
2,"Villa near Vienna Project, Outside Vienna, Aus...",Emil Hoppe,7605,"(Austrian, 1876–1957)",(Austrian),(1876),(1957),(Male),1903,"Graphite, pen, color pencil, ink, and gouache ...",...,http://www.moma.org/media/W1siZiIsIjk4Il0sWyJw...,NaN,NaN,NaN,34.3000,NaN,NaN,31.8000,NaN,NaN
3,"The Manhattan Transcripts Project, New York, N...",Bernard Tschumi,7056,"(French and Swiss, born Switzerland 1944)",(),(1944),(0),(Male),1980,Photographic reproduction with colored synthet...,...,http://www.moma.org/media/W1siZiIsIjEyNCJdLFsi...,NaN,NaN,NaN,50.8000,NaN,NaN,50.8000,NaN,NaN
4,"Villa, project, outside Vienna, Austria, Exter...",Emil Hoppe,7605,"(Austrian, 1876–1957)",(Austrian),(1876),(1957),(Male),1903,"Graphite, color pencil, ink, and gouache on tr...",...,http://www.moma.org/media/W1siZiIsIjEyNiJdLFsi...,NaN,NaN,NaN,38.4000,NaN,NaN,19.1000,NaN,NaN


In [4]:
print(raw_data.columns)
raw_data.head()

Index(['Title', 'Artist', 'ConstituentID', 'ArtistBio', 'Nationality',
       'BeginDate', 'EndDate', 'Gender', 'Date', 'Medium', 'Dimensions',
       'CreditLine', 'AccessionNumber', 'Classification', 'Department',
       'DateAcquired', 'Cataloged', 'ObjectID', 'URL', 'ThumbnailURL',
       'Circumference (cm)', 'Depth (cm)', 'Diameter (cm)', 'Height (cm)',
       'Length (cm)', 'Weight (kg)', 'Width (cm)', 'Seat Height (cm)',
       'Duration (sec.)'],
      dtype='object')


,Title,Artist,ConstituentID,ArtistBio,Nationality,BeginDate,EndDate,Gender,Date,Medium,...,ThumbnailURL,Circumference (cm),Depth (cm),Diameter (cm),Height (cm),Length (cm),Weight (kg),Width (cm),Seat Height (cm),Duration (sec.)
0,"Ferdinandsbrücke Project, Vienna, Austria, Ele...",Otto Wagner,6210,"(Austrian, 1841–1918)",(Austrian),(1841),(1918),(Male),1896,Ink and cut-and-pasted painted pages on paper,...,http://www.moma.org/media/W1siZiIsIjU5NDA1Il0s...,NaN,NaN,NaN,48.6000,NaN,NaN,168.9000,NaN,NaN
1,"City of Music, National Superior Conservatory ...",Christian de Portzamparc,7470,"(French, born 1944)",(French),(1944),(0),(Male),1987,Paint and colored pencil on print,...,http://www.moma.org/media/W1siZiIsIjk3Il0sWyJw...,NaN,NaN,NaN,40.6401,NaN,NaN,29.8451,NaN,NaN
2,"Villa near Vienna Project, Outside Vienna, Aus...",Emil Hoppe,7605,"(Austrian, 1876–1957)",(Austrian),(1876),(1957),(Male),1903,"Graphite, pen, color pencil, ink, and gouache ...",...,http://www.moma.org/media/W1siZiIsIjk4Il0sWyJw...,NaN,NaN,NaN,34.3000,NaN,NaN,31.8000,NaN,NaN
3,"The Manhattan Transcripts Project, New York, N...",Bernard Tschumi,7056,"(French and Swiss, born Switzerland 1944)",(),(1944),(0),(Male),1980,Photographic reproduction with colored synthet...,...,http://www.moma.org/media/W1siZiIsIjEyNCJdLFsi...,NaN,NaN,NaN,50.8000,NaN,NaN,50.8000,NaN,NaN
4,"Villa, project, outside Vienna, Austria, Exter...",Emil Hoppe,7605,"(Austrian, 1876–1957)",(Austrian),(1876),(1957),(Male),1903,"Graphite, color pencil, ink, and gouache on tr...",...,http://www.moma.org/media/W1siZiIsIjEyNiJdLFsi...,NaN,NaN,NaN,38.4000,NaN,NaN,19.1000,NaN,NaN


# Process Data

In [5]:
# Select Columns.
artworks = raw_data[['Artist', 'Nationality', 'Gender', 'Date', 'Department',
                    'DateAcquired', 'URL', 'ThumbnailURL', 'Height (cm)', 'Width (cm)']]

In [6]:
# To avoid setting with copy warning...
artworks = artworks.copy()

In [7]:
# Convert URL's to booleans.
artworks['URL'] = artworks['URL'].notnull()
artworks['ThumbnailURL'] = artworks['ThumbnailURL'].notnull()

In [8]:
# Drop films and some other tricky rows.
artworks = artworks[artworks['Department'] != 'Film']
artworks = artworks[artworks['Department'] != 'Media and Performance Art']
artworks = artworks[artworks['Department'] != 'Fluxus Collection']

In [9]:
# Drop missing data.
artworks = artworks.dropna()

In [10]:
# View dataframe.
artworks.head()

,Artist,Nationality,Gender,Date,Department,DateAcquired,URL,ThumbnailURL,Height (cm),Width (cm)
0,Otto Wagner,(Austrian),(Male),1896,Architecture & Design,1996-04-09,True,True,48.6000,168.9000
1,Christian de Portzamparc,(French),(Male),1987,Architecture & Design,1995-01-17,True,True,40.6401,29.8451
2,Emil Hoppe,(Austrian),(Male),1903,Architecture & Design,1997-01-15,True,True,34.3000,31.8000
3,Bernard Tschumi,(),(Male),1980,Architecture & Design,1995-01-17,True,True,50.8000,50.8000
4,Emil Hoppe,(Austrian),(Male),1903,Architecture & Design,1997-01-15,True,True,38.4000,19.1000


## Building the Model

Use Multi-Layer Perceptron modeling (MLP) to see if we can classify the department a piece should go into using everything but the department name.

First, let's ensure correct typing for our data & do other cleaning.

In [11]:
# Get data types.
artworks.dtypes

Artist           object
Nationality      object
Gender           object
Date             object
Department       object
DateAcquired     object
URL                bool
ThumbnailURL       bool
Height (cm)     float64
Width (cm)      float64
dtype: object

In [12]:
# DateAcquired column is an object, let's transform that to datetime object
# & add a feature for just the year the artwork was acquired.
artworks['DateAcquired'] = pd.to_datetime(artworks.DateAcquired)
artworks['YearAcquired'] = artworks.DateAcquired.dt.year
artworks['YearAcquired'].dtype

dtype('int64')

In [13]:
# Remove multiple nationalities, genders, and artists.
artworks.loc[artworks['Gender'].str.contains('\) \('), 'Gender'] = '\(multiple_persons\)'
artworks.loc[artworks['Nationality'].str.contains('\) \('), 'Nationality'] = '\(multiple_nationalities\)'
artworks.loc[artworks['Artist'].str.contains(','), 'Artist'] = 'Multiple_Artists'

In [14]:
artworks.head()

,Artist,Nationality,Gender,Date,Department,DateAcquired,URL,ThumbnailURL,Height (cm),Width (cm),YearAcquired
0,Otto Wagner,(Austrian),(Male),1896,Architecture & Design,1996-04-09,True,True,48.6000,168.9000,1996
1,Christian de Portzamparc,(French),(Male),1987,Architecture & Design,1995-01-17,True,True,40.6401,29.8451,1995
2,Emil Hoppe,(Austrian),(Male),1903,Architecture & Design,1997-01-15,True,True,34.3000,31.8000,1997
3,Bernard Tschumi,(),(Male),1980,Architecture & Design,1995-01-17,True,True,50.8000,50.8000,1995
4,Emil Hoppe,(Austrian),(Male),1903,Architecture & Design,1997-01-15,True,True,38.4000,19.1000,1997


In [15]:
# Convert dates to start date, cutting down number of distinct examples.
artworks['Date'] = pd.Series(artworks.Date.str.extract(
    '([0-9]{4})', expand=False))[:-1]

In [16]:
# Final column drops and NA drop.
X = artworks.drop(['Department', 'DateAcquired', 'Artist', 'Nationality', 'Date'], 1)

In [17]:
# Create dummies separately.
artists = pd.get_dummies(artworks.Artist)
nationalities = pd.get_dummies(artworks.Nationality)
dates = pd.get_dummies(artworks.Date)

In [18]:
# Concat with other variables, but artists slows this way down so we'll 
# keep it out for now.
X = pd.get_dummies(X, sparse = True)
X = pd.concat([X, nationalities, dates], axis = 1)
Y = artworks.Department

### Prep complete, let's build the model.
Neural networks are computationally intensive, this may take several minutes to run.

In [19]:
"""
# Import the model.
from sklearn.neural_network import MLPClassifier

# Establish and fit the model, with a single, 1000 perceptron layer.
mlp = MLPClassifier(hidden_layer_sizes = (1000,))
mlp.fit(X, Y)
mlp.score(X, Y)
Y.value_counts() / len(Y)
from sklearn.model_selection import cross_val_score
cross_val_score(mlp, X, Y, cv = 5)
"""

### Results

Model seems to overfit, though there is still remaining performance when validated with cross validation. This occurs in neural networks that aren't given enough data for the number of features present. 

Increasing the layer size will increase runtime. 

Note we created bools for artist's name but left them out. Both of the above points are the reason for that - it would take much longer to run and it would be much more prone to overfitting.

### Model Parameters
1. Hidden Layer Size
2. Alpha
3. Activation Function

We included 1 parameter: **Hidden Layer Size**. This tells us how many and how big to make our layers. Pass in a tuple that specifies each layer's size. Our network is 1000 neurons wide and 1 layer. (100, 4, ) would create a network with 2 layers, one 100 wide and the other 4. 

How many layers? Determined by computational resources & cross validation searching for convergence. It is generally less than the number of input variables you have. 

You can also set an **Alpha**, in which NN like this use as a regularization parameter that penalizes large coefficients just like we discussed in the advanced regression section. Alpha scales that penalty. 

**Activation Function**: determines whether the output from an individual perceptron is binary or continuous. By default, this is a 'relu', or 'rectified linear unit function' function. In this exercise, we used this binary function, but we discussed the *sigmoid* as a reasonable alternative. The *sigmoid* (called 'logistic' by SKLearn because it's a 'logistic sigmoid function') allows for continuous variables btwn 0 and 1, which allows for a more nuanced model. It does come at the cost of increased computational complexity. 

### Experiment with different hidden layer structures

Try on a subset to improve runtime. 

In [27]:
artworks.shape

(107160, 11)

In [37]:
# Take a subset of the data.
X_sample = X.sample(frac = 0.1)
Y_sample = Y.sample(frac = 0.1)

"""
# hidden_layer_sizes : tuple, length = n_layers - 2, default (100,)
# alpha : float, optional, default 0.0001
# activation : {‘identity’, ‘logistic’, ‘tanh’, ‘relu’}, default ‘relu’
"""

# Try the different layers.
def runMLP(layerSize, alphaSize, activationFunct):
    print('\nBegin\n')
    start_time = time.time()
    from sklearn.neural_network import MLPClassifier
    mlp = MLPClassifier(hidden_layer_sizes=layerSize, alpha=alphaSize, activation=activationFunct)
    mlp.fit(X_sample, Y_sample)
    print(mlp.score(X_sample, Y_sample))
    from sklearn.model_selection import cross_val_score
    print(cross_val_score(mlp, X_sample, Y_sample, cv=5))
    print('Time taken: {} seconds'.format('%.1f' % (time.time() - start_time)))
    
# Default.
print(runMLP(100, 0.0001, 'relu'))

# Tuning.
runMLP(100, 0.00001, 'relu')
runMLP(100, 0.001, 'relu')
runMLP(100, 0.01, 'relu')




#layers_list = [(50,), (100,), (150,)]
#alphas_list = [0.00001, 0.0001, 0.001]
#activations_list = ['identity', 'relu', 'logistic']
#layers_list = [(50), (100), (150)]
#alphas_list = [0.00001, 0.0001, 0.001]
#activations_list = ['identity', 'relu', 'logistic']
#for a, b, c in layers_list, alphas_list, activations_list:
#    runMLP(a, b, c)


Begin

0.22685703620754014
[0.62424242 0.5972028  0.62424242 0.62540869 0.61915888]
Time taken: 5.7 seconds
Defaults: None

Begin

0.5459126539753639
[0.62284382 0.62470862 0.62331002 0.62540869 0.51121495]
Time taken: 6.0 seconds

Begin

0.6192609182530795
[0.62424242 0.62470862 0.62424242 0.6235404  0.59813084]
Time taken: 6.5 seconds

Begin

0.06896229936543487
[0.62470862 0.62004662 0.34405594 0.22839794 0.62570093]
Time taken: 6.9 seconds
